In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from einops import rearrange, reduce, repeat

In [2]:
class FC(nn.Module):
    def __init__(self):
        super(FC, self).__init__()
        self.hidden1 = nn.Linear(3, 4)
        # self.hidden2 = nn.Linear(4, 1)

    def forward(self, x):
        x = self.hidden1(x)
        # x = self.hidden2(x)
        return x

model = FC()
print("training parameter:", sum(p.numel() for p in model.parameters() if p.requires_grad))


# 3 * 4 + 4 = 16
# 4 * 1 + 1 = 5
# 21 = 16 + 5
input_tensor = torch.randn(50, 1, 3)
output = model(input_tensor)
print(output.shape)

training parameter: 16
torch.Size([50, 1, 4])


In [3]:
class CONV(nn.Module):
    def __init__(self):
        super(CONV, self).__init__()
        self.layer1 = nn.Conv3d(in_channels = 1, out_channels = 8, kernel_size = (3, 3, 8), padding = (1, 1, 0), stride = 1, bias = False)
#         self.layer2 = nn.Conv3d(in_channels = 1, out_channels = 8, kernel_size = (3, 3, 8), padding = (1, 1, 0), stride = 1, bias = False)

    def forward(self, x):
        out = self.layer1(x)
#         x2 = rearrange(x1, 'b k h w c -> b c h w k')
#         x3 = self.layer1(x2)
#         out = rearrange(x3, 'b k h w c -> b c h w k')
        return out

model = CONV()
print("training parameter:", sum(p.numel() for p in model.parameters() if p.requires_grad))

input_tensor = torch.randn(50, 1, 6, 7, 8)
output = model(input_tensor)
print(output.shape)

# https://netron.app/
onnx_file_path = "model.onnx"
torch.onnx.export(model, input_tensor, onnx_file_path, verbose=True, opset_version=11)
print(f"Model has been saved to {onnx_file_path}")

training parameter: 576
torch.Size([50, 8, 6, 7, 1])
Model has been saved to model.onnx


In [4]:
class CONV(nn.Module):
    def __init__(self):
        super(CONV, self).__init__()
        self.layer = nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = (3, 3), bias = False)

    def forward(self, x):
        out = self.layer1(x)
        return out

model = CONV()
print("training parameter:", sum(p.numel() for p in model.parameters() if p.requires_grad))

# input_tensor = torch.randn(50, 1, 6, 7, 8)
# output = model(input_tensor)
# print(output.shape)

# # https://netron.app/
# onnx_file_path = "model.onnx"
# torch.onnx.export(model, input_tensor, onnx_file_path, verbose=True, opset_version=11)
# print(f"Model has been saved to {onnx_file_path}")

training parameter: 9
